# Download the datafile [here](https://www.dropbox.com/s/0r508fymxtnepuz/all_data.hdf?dl=0) and place in the same folder as this notebook.

# [and here](https://www.dropbox.com/s/cyaxqukd89sqfe6/all_data_square.hdf?dl=0) for the data with a squared wire

In [ ]:
from types import SimpleNamespace
import numpy as np
import holoviews as hv
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') 
pd.set_option('display.max_columns', None)

def dims(df):
    return SimpleNamespace(phi=hv.Dimension(r'$\phi$', unit='rad'),
                           phi_c=hv.Dimension(r'$\phi_c$', unit='rad'),
                           I=hv.Dimension('$I$', unit=r'meV / $\Phi_0$'),
                           I_c=hv.Dimension('$I_c$', unit=r'meV / $\Phi_0$'),
                           B=hv.Dimension('$B$', unit='T', values=df['B_x'].unique().tolist()),
                           T=hv.Dimension('$T$', unit='K', values=df['T'].unique().tolist()), 
                           L=hv.Dimension('$L$', unit='nm', values=df['L'].unique().tolist()), 
                           g=hv.Dimension('$g$-factor', values=df['g'].unique().tolist()), 
                           alpha=hv.Dimension(r'$\alpha$', unit='mev nm', values=df['alpha'].unique().tolist()), 
                           mu=hv.Dimension(r'$\mu$', unit='meV', values=df['mu'].unique().tolist()), 
                           a=hv.Dimension('$a$', unit='nm', values=df['a'].unique().tolist()),
                           orbital=hv.Dimension('orbital', values=[True, False]),
                           leads=hv.Dimension('leads', values=[True, False]),
                           zoom=hv.Dimension('zoom', values=[5, 10, 25, 50, 75, 100, 200]))

def dynamic_map(fname, title=None):
    df = pd.read_hdf(fname)
    d = dims(df)
#     d.T = hv.Dimension('$T$', unit='K', values=[1])
    gb = df.groupby(['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a'])
    groups = dict(list(gb))

    if title is None:
        title = fname

    def current_curve(T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
        return hv.Path((gr['B_x'], gr['current_c']), kdims=[d.B, d.I_c], label=title)[:, 0:zoom]

    def phase_curve(T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
        return hv.Path((gr['B_x'], gr['phase_c']), kdims=[d.B, d.phi_c], label=title)[:, -np.pi:np.pi]

    def current_phase(B_x, T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
        gr = gr[gr['B_x'] == B_x]
        phases = np.array(gr['phases'])[0]
        currents = np.array(gr['currents'])[0]
        return (hv.Path((phases, currents), kdims=[d.phi, d.I], label=title)[:, -zoom:zoom] *
                hv.Scatter((gr['phase_c'], gr['current_c'])))

    kdims = [d.T, d.L, d.leads, d.orbital, d.g, d.alpha, d.mu, d.a, d.zoom]

    return (hv.DynamicMap(current_curve, kdims=kdims),
            hv.DynamicMap(phase_curve, kdims=kdims),
            hv.DynamicMap(current_phase, kdims=[d.B] + kdims))

# Critical current and phase, $B_x$ vs $I_c$ and $B_x$ vs $\phi_c$

In [ ]:
circular_A_incorrect = dynamic_map('data/all_data_circular_incorrect_vector_potential.hdf')

square_A_in_SM = dynamic_map('data/all_data_square_A_in_SC.hdf', r'$A$ everywhere')
square_A_in_SM_and_SC = dynamic_map('data/all_data_square_no_A_in_SC.hdf', r'$A$ only in SM')

In [ ]:
%%opts Path [show_legend=True]

square_A_in_SM[0] * square_A_in_SM_and_SC[0] + square_A_in_SM[1] * square_A_in_SM_and_SC[1]

# Current phase relation, $\phi$ vs $I$
Adjust `zoom` slider to change the `y`-range of the plot.

In [ ]:
# hm + currents[key] * hv.HoloMap({B: hv.VLine(B) for B in df['B_x'].unique()}, kdims=[d.B])
square_A_in_SM[2] * square_A_in_SM_and_SC[2]

# Gate

In [ ]:
%%opts Image [colorbar=True aspect='square']
df = pd.read_hdf('data/all_gate_data_current_as_function_of_gate_and_B_x.hdf')
d = dims(df)
d.V = hv.Dimension('$V$', unit='meV')
inds = ['gate_size', 'T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a']
kdims = ['gate_size', d.T, d.L, 'infinite leads', 'orbital', d.g, d.alpha, d.mu, d.a]

gb = df.groupby(inds)
bnds = (df['B_x'].min(), df['Vs'].min(), df['B_x'].max(), df['Vs'].max())
gate_sizes = hv.HoloMap({key: hv.Image(np.rot90(val['current_c'].reshape(100, -1)),
                                       bounds=bnds, kdims=[d.B, d.V], vdims=[d.I_c])
                         for key, val in gb}, kdims=kdims)


gate_sizes

# Debug

In [ ]:
dm1 = hv.DynamicMap(lambda x, y: hv.Curve(np.random.rand(10)), kdims=[hv.Dimension('x', values=[0, 1, 2]), hv.Dimension('y', values=[5, 7])])
dm2 = hv.DynamicMap(lambda x, y: hv.Curve(np.random.rand(10)), kdims=[hv.Dimension('x', values=[3]), hv.Dimension('y', values=[5, 7])])
dm2 * dm1